In [1]:
import os

RAW_DIR = "raw_LIV"

# Collect files
files = sorted([f for f in os.listdir(RAW_DIR) if f.endswith(".csv")])

# Extract unique device prefixes (before _LIV)
devices = sorted(set(f.split("_")[0] for f in files))

# Create mapping: deviceA -> device001, etc.
mapping = {
    dev: f"device{str(i+1).zfill(3)}"
    for i, dev in enumerate(devices)
}

print("Renaming mapping:")
for k, v in mapping.items():
    print(f"{k} → {v}")

# Rename files
for f in files:
    old_dev = f.split("_")[0]
    new_dev = mapping[old_dev]
    new_name = f.replace(old_dev, new_dev)

    os.rename(
        os.path.join(RAW_DIR, f),
        os.path.join(RAW_DIR, new_name)
    )

print("Renaming completed.")

Renaming mapping:
deviceA → device001
deviceB → device002
deviceC → device003
deviceD → device004
deviceE → device005
deviceF → device006
deviceG → device007
deviceH → device008
deviceI → device009
deviceJ → device010
deviceK → device011
deviceL → device012
deviceM → device013
deviceN → device014
deviceO → device015
deviceP → device016
deviceQ → device017
deviceR → device018
deviceS → device019
deviceT → device020
deviceU → device021
Renaming completed.


In [3]:
import os
import re

RAW_DIR = "raw_LIV"

files = sorted(os.listdir(RAW_DIR))

# Keep only LIV1 / LIV2 files (with or without extension)
liv_files = [f for f in files if ("LIV1" in f or "LIV2" in f)]

# Detect existing device numbers
existing_nums = []
for f in liv_files:
    m = re.match(r"(device\d{3})_", f)
    if m:
        existing_nums.append(int(m.group(1).replace("device", "")))

start_idx = max(existing_nums) + 1 if existing_nums else 1

# Extract prefixes before first _
prefixes = sorted(set(f.split("_")[0] for f in liv_files))

# Filter out already standardised prefixes (device###)
new_prefixes = [p for p in prefixes if not re.match(r"device\d{3}", p)]

# Create mapping
mapping = {
    old: f"device{str(start_idx + i).zfill(3)}"
    for i, old in enumerate(new_prefixes)
}

print("Renaming mapping:")
for k, v in mapping.items():
    print(f"{k} → {v}")

# Rename files
for f in liv_files:
    old_prefix = f.split("_")[0]
    if old_prefix in mapping:
        new_name = f.replace(old_prefix, mapping[old_prefix])
        os.rename(
            os.path.join(RAW_DIR, f),
            os.path.join(RAW_DIR, new_name)
        )

print("Renaming completed.")

Renaming mapping:
Renaming completed.


In [4]:
import os, re

RAW_DIR = "raw_LIV"
all_files = sorted(os.listdir(RAW_DIR))

liv_files = [f for f in all_files if re.search(r"LIV[12]", f, re.IGNORECASE)]
not_standard = [f for f in liv_files if not re.match(r"device\d{3}_", f, re.IGNORECASE)]

print("Total LIV files:", len(liv_files))
print("Not yet standard (to rename):", len(not_standard))
print("\nExamples:")
for f in not_standard[:20]:
    print("  ", f)

Total LIV files: 133
Not yet standard (to rename): 91

Examples:
   Mazhar  Abbas_081324-Angle-Pressure exp_Batch1_sample3_x_6_scan_1.liv1
   Mazhar  Abbas_081324-Angle-Pressure exp_Batch1_sample3_x_6_scan_1.liv2
   Mazhar  Abbas_081324-Angle-Pressure exp_Batch1_sample3_x_7_scan_0.liv1
   Mazhar  Abbas_081324-Angle-Pressure exp_Batch1_sample3_x_7_scan_0.liv2
   Mazhar  Abbas_081324-Angle-Pressure exp_Batch1_sample3_x_7_scan_1.liv1
   Mazhar  Abbas_081324-Angle-Pressure exp_Batch1_sample3_x_7_scan_1.liv2
   Mazhar  Abbas_081324-Angle-Pressure exp_Batch1_sample3_x_8_scan_0.liv1
   Mazhar  Abbas_081324-Angle-Pressure exp_Batch1_sample3_x_8_scan_0.liv2
   Mazhar  Abbas_081324-Angle-Pressure exp_Batch1_sample3_x_8_scan_1.liv1
   Mazhar  Abbas_081324-Angle-Pressure exp_Batch1_sample3_x_8_scan_1.liv2
   Mazhar Abbas_01032025_WBGPVK_1.5_Sample11_x_6_scan_10.liv1
   Mazhar Abbas_01032025_WBGPVK_1.5_Sample11_x_6_scan_10.liv2
   Mazhar Abbas_01032025_WBGPVK_1.5_Sample11_x_6_scan_7.liv1
   Mazhar 

In [5]:
import os, re

RAW_DIR = "raw_LIV"
files = sorted(os.listdir(RAW_DIR))

# Only LIV1/LIV2 files (any extension or none)
liv_files = [f for f in files if re.search(r"LIV[12]", f, re.IGNORECASE)]

# Find highest existing device number (device001_..., device021_..., etc.)
existing_nums = []
for f in liv_files:
    m = re.match(r"device(\d{3})_", f, re.IGNORECASE)
    if m:
        existing_nums.append(int(m.group(1)))

start_idx = (max(existing_nums) + 1) if existing_nums else 1

# Extract the "original prefix" = everything before LIV1/LIV2
def get_prefix(fname: str) -> str:
    # grabs text before LIV1/LIV2 (handles underscores, spaces, hyphens, etc.)
    m = re.match(r"^(.*?)(?:[_\s-]*LIV[12])", fname, re.IGNORECASE)
    return m.group(1).strip() if m else fname.split("_")[0].strip()

# Build list of prefixes that are NOT already standardized
prefixes = sorted(set(get_prefix(f) for f in liv_files))
new_prefixes = [p for p in prefixes if not re.match(r"^device\d{3}$", p, re.IGNORECASE)]

mapping = {old: f"device{str(start_idx + i).zfill(3)}" for i, old in enumerate(new_prefixes)}

print("Renaming mapping (new only):")
for k, v in mapping.items():
    print(f"{k}  →  {v}")

# Rename
for f in liv_files:
    pref = get_prefix(f)
    if pref in mapping:
        new_name = re.sub(re.escape(pref), mapping[pref], f, count=1)
        src = os.path.join(RAW_DIR, f)
        dst = os.path.join(RAW_DIR, new_name)

        if os.path.exists(dst):
            raise FileExistsError(f"Target already exists: {new_name}")

        os.rename(src, dst)

print("Renaming completed.")

Renaming mapping (new only):
Mazhar  Abbas_081324-Angle-Pressure exp_Batch1_sample3_x_6_scan_1.  →  device022
Mazhar  Abbas_081324-Angle-Pressure exp_Batch1_sample3_x_7_scan_0.  →  device023
Mazhar  Abbas_081324-Angle-Pressure exp_Batch1_sample3_x_7_scan_1.  →  device024
Mazhar  Abbas_081324-Angle-Pressure exp_Batch1_sample3_x_8_scan_0.  →  device025
Mazhar  Abbas_081324-Angle-Pressure exp_Batch1_sample3_x_8_scan_1.  →  device026
Mazhar Abbas_01032025_WBGPVK_1.5_Sample11_x_6_scan_10.  →  device027
Mazhar Abbas_01032025_WBGPVK_1.5_Sample11_x_6_scan_7.  →  device028
Mazhar Abbas_01032025_WBGPVK_1.5_Sample11_x_7_scan_1.  →  device029
Mazhar Abbas_01032025_WBGPVK_1.5_Sample11_x_7_scan_2.  →  device030
Mazhar Abbas_01032025_WBGPVK_1.5_Sample11_x_7_scan_3.  →  device031
Mazhar Abbas_01032025_WBGPVK_1.5_Sample11_x_7_scan_4.  →  device032
Mazhar Abbas_01032025_WBGPVK_1.5_Sample11_x_7_scan_5.  →  device033
Mazhar Abbas_01032025_WBGPVK_1.5_Sample11_x_7_scan_6.  →  device034
Mazhar Abbas_01032025

In [6]:
from collections import Counter
import os, re

RAW_DIR = "raw_LIV"
files = [f for f in os.listdir(RAW_DIR) if re.search(r"LIV[12]", f, re.IGNORECASE)]
counts = Counter(re.match(r"(device\d{3})_", f, re.IGNORECASE).group(1) for f in files if re.match(r"(device\d{3})_", f, re.IGNORECASE))

bad = {k:v for k,v in counts.items() if v != 2}
print("Total devices:", len(counts))
print("Devices not having exactly 2 files (LIV1+LIV2):", bad)

Total devices: 21
Devices not having exactly 2 files (LIV1+LIV2): {}


In [7]:
import os, re

RAW_DIR = "raw_LIV"

files = os.listdir(RAW_DIR)

for f in files:
    # Match patterns like: device022liv1, device022liv2, device022_LIV1, device022 LIV1, etc.
    m = re.match(r"(device\d{3})[_\s-]*(liv[12])(\.csv)?$", f, re.IGNORECASE)
    if m:
        dev = m.group(1).lower()
        liv = m.group(2).upper()
        new_name = f"{dev}_{liv}.csv"

        if f != new_name:
            os.rename(
                os.path.join(RAW_DIR, f),
                os.path.join(RAW_DIR, new_name)
            )
            print(f"Fixed: {f} → {new_name}")

print("Normalization complete.")

Fixed: device022liv1 → device022_LIV1.csv
Fixed: device022liv2 → device022_LIV2.csv
Fixed: device023liv1 → device023_LIV1.csv
Fixed: device023liv2 → device023_LIV2.csv
Fixed: device024liv1 → device024_LIV1.csv
Fixed: device024liv2 → device024_LIV2.csv
Fixed: device025liv1 → device025_LIV1.csv
Fixed: device025liv2 → device025_LIV2.csv
Fixed: device026liv1 → device026_LIV1.csv
Fixed: device026liv2 → device026_LIV2.csv
Fixed: device027liv1 → device027_LIV1.csv
Fixed: device027liv2 → device027_LIV2.csv
Fixed: device028liv1 → device028_LIV1.csv
Fixed: device028liv2 → device028_LIV2.csv
Fixed: device029liv1 → device029_LIV1.csv
Fixed: device030liv1 → device030_LIV1.csv
Fixed: device030liv2 → device030_LIV2.csv
Fixed: device031liv1 → device031_LIV1.csv
Fixed: device031liv2 → device031_LIV2.csv
Fixed: device032liv1 → device032_LIV1.csv
Fixed: device032liv2 → device032_LIV2.csv
Fixed: device033liv1 → device033_LIV1.csv
Fixed: device033liv2 → device033_LIV2.csv
Fixed: device034liv1 → device034_L

In [8]:
from collections import Counter
import os, re

files = [f for f in os.listdir("raw_LIV") if re.search(r"LIV[12]\.csv$", f, re.IGNORECASE)]
counts = Counter(re.match(r"(device\d{3})_", f).group(1) for f in files)

bad = {k:v for k,v in counts.items() if v != 2}

print("Total devices:", len(counts))
print("Devices not having exactly 2 scans:", bad)

Total devices: 67
Devices not having exactly 2 scans: {'device029': 1}


In [9]:
import os, re

RAW_DIR = "raw_LIV"
all_files = sorted(os.listdir(RAW_DIR))

liv_files = [f for f in all_files if re.search(r"LIV[12]", f, re.IGNORECASE)]
not_standard = [f for f in liv_files if not re.match(r"device\d{3}_", f, re.IGNORECASE)]

print("Total LIV files:", len(liv_files))
print("Not yet standard (to rename):", len(not_standard))
print("\nExamples:")
for f in not_standard[:20]:
    print("  ", f)

Total LIV files: 212
Not yet standard (to rename): 78

Examples:
   Mazhar Abbas_121724_WBD_4Pacz_sample1_x_1_scan_0.liv1
   Mazhar Abbas_121724_WBD_4Pacz_sample1_x_1_scan_0.liv2
   Mazhar Abbas_121724_WBD_4Pacz_sample1_x_1_scan_1.liv1
   Mazhar Abbas_121724_WBD_4Pacz_sample1_x_1_scan_1.liv2
   Mazhar Abbas_121724_WBD_4Pacz_sample1_x_1_scan_2.liv1
   Mazhar Abbas_121724_WBD_4Pacz_sample1_x_1_scan_2.liv2
   Mazhar Abbas_121724_WBD_4Pacz_sample1_x_1_scan_3.liv1
   Mazhar Abbas_121724_WBD_4Pacz_sample1_x_1_scan_3.liv2
   Mazhar Abbas_121724_WBD_4Pacz_sample1_x_2_scan_0.liv1
   Mazhar Abbas_121724_WBD_4Pacz_sample1_x_2_scan_0.liv2
   Mazhar Abbas_121724_WBD_4Pacz_sample1_x_2_scan_1.liv1
   Mazhar Abbas_121724_WBD_4Pacz_sample1_x_2_scan_1.liv2
   Mazhar Abbas_121724_WBD_4Pacz_sample1_x_2_scan_2.liv1
   Mazhar Abbas_121724_WBD_4Pacz_sample1_x_2_scan_2.liv2
   Mazhar Abbas_121724_WBD_4Pacz_sample1_x_2_scan_3.liv1
   Mazhar Abbas_121724_WBD_4Pacz_sample1_x_2_scan_3.liv2
   Mazhar Abbas_121724_

In [10]:
import os, re

RAW_DIR = "raw_LIV"
files = sorted(os.listdir(RAW_DIR))

# Only LIV1/LIV2 files (any extension or none)
liv_files = [f for f in files if re.search(r"LIV[12]", f, re.IGNORECASE)]

# Find highest existing device number (device001_..., device021_..., etc.)
existing_nums = []
for f in liv_files:
    m = re.match(r"device(\d{3})_", f, re.IGNORECASE)
    if m:
        existing_nums.append(int(m.group(1)))

start_idx = (max(existing_nums) + 1) if existing_nums else 1

# Extract the "original prefix" = everything before LIV1/LIV2
def get_prefix(fname: str) -> str:
    # grabs text before LIV1/LIV2 (handles underscores, spaces, hyphens, etc.)
    m = re.match(r"^(.*?)(?:[_\s-]*LIV[12])", fname, re.IGNORECASE)
    return m.group(1).strip() if m else fname.split("_")[0].strip()

# Build list of prefixes that are NOT already standardized
prefixes = sorted(set(get_prefix(f) for f in liv_files))
new_prefixes = [p for p in prefixes if not re.match(r"^device\d{3}$", p, re.IGNORECASE)]

mapping = {old: f"device{str(start_idx + i).zfill(3)}" for i, old in enumerate(new_prefixes)}

print("Renaming mapping (new only):")
for k, v in mapping.items():
    print(f"{k}  →  {v}")

# Rename
for f in liv_files:
    pref = get_prefix(f)
    if pref in mapping:
        new_name = re.sub(re.escape(pref), mapping[pref], f, count=1)
        src = os.path.join(RAW_DIR, f)
        dst = os.path.join(RAW_DIR, new_name)

        if os.path.exists(dst):
            raise FileExistsError(f"Target already exists: {new_name}")

        os.rename(src, dst)

print("Renaming completed.")

Renaming mapping (new only):
Mazhar Abbas_121724_WBD_4Pacz_sample1_x_1_scan_0.  →  device068
Mazhar Abbas_121724_WBD_4Pacz_sample1_x_1_scan_1.  →  device069
Mazhar Abbas_121724_WBD_4Pacz_sample1_x_1_scan_2.  →  device070
Mazhar Abbas_121724_WBD_4Pacz_sample1_x_1_scan_3.  →  device071
Mazhar Abbas_121724_WBD_4Pacz_sample1_x_2_scan_0.  →  device072
Mazhar Abbas_121724_WBD_4Pacz_sample1_x_2_scan_1.  →  device073
Mazhar Abbas_121724_WBD_4Pacz_sample1_x_2_scan_2.  →  device074
Mazhar Abbas_121724_WBD_4Pacz_sample1_x_2_scan_3.  →  device075
Mazhar Abbas_121724_WBD_4Pacz_sample1_x_3_scan_0.  →  device076
Mazhar Abbas_121724_WBD_4Pacz_sample1_x_3_scan_1.  →  device077
Mazhar Abbas_121724_WBD_4Pacz_sample1_x_3_scan_2.  →  device078
Mazhar Abbas_121724_WBD_4Pacz_sample1_x_3_scan_3.  →  device079
Mazhar Abbas_121724_WBD_4Pacz_sample1_x_4_scan_0.  →  device080
Mazhar Abbas_121724_WBD_4Pacz_sample1_x_4_scan_1.  →  device081
Mazhar Abbas_121724_WBD_4Pacz_sample1_x_4_scan_2.  →  device082
Mazhar Abba

In [11]:
from collections import Counter
import os, re

RAW_DIR = "raw_LIV"
files = [f for f in os.listdir(RAW_DIR) if re.search(r"LIV[12]", f, re.IGNORECASE)]
counts = Counter(re.match(r"(device\d{3})_", f, re.IGNORECASE).group(1) for f in files if re.match(r"(device\d{3})_", f, re.IGNORECASE))

bad = {k:v for k,v in counts.items() if v != 2}
print("Total devices:", len(counts))
print("Devices not having exactly 2 files (LIV1+LIV2):", bad)

Total devices: 67
Devices not having exactly 2 files (LIV1+LIV2): {}


In [12]:
import os, re

RAW_DIR = "raw_LIV"

files = os.listdir(RAW_DIR)

for f in files:
    # Match patterns like: device022liv1, device022liv2, device022_LIV1, device022 LIV1, etc.
    m = re.match(r"(device\d{3})[_\s-]*(liv[12])(\.csv)?$", f, re.IGNORECASE)
    if m:
        dev = m.group(1).lower()
        liv = m.group(2).upper()
        new_name = f"{dev}_{liv}.csv"

        if f != new_name:
            os.rename(
                os.path.join(RAW_DIR, f),
                os.path.join(RAW_DIR, new_name)
            )
            print(f"Fixed: {f} → {new_name}")

print("Normalization complete.")

Fixed: device068liv1 → device068_LIV1.csv
Fixed: device068liv2 → device068_LIV2.csv
Fixed: device069liv1 → device069_LIV1.csv
Fixed: device069liv2 → device069_LIV2.csv
Fixed: device070liv1 → device070_LIV1.csv
Fixed: device070liv2 → device070_LIV2.csv
Fixed: device071liv1 → device071_LIV1.csv
Fixed: device071liv2 → device071_LIV2.csv
Fixed: device072liv1 → device072_LIV1.csv
Fixed: device072liv2 → device072_LIV2.csv
Fixed: device073liv1 → device073_LIV1.csv
Fixed: device073liv2 → device073_LIV2.csv
Fixed: device074liv1 → device074_LIV1.csv
Fixed: device074liv2 → device074_LIV2.csv
Fixed: device075liv1 → device075_LIV1.csv
Fixed: device075liv2 → device075_LIV2.csv
Fixed: device076liv1 → device076_LIV1.csv
Fixed: device076liv2 → device076_LIV2.csv
Fixed: device077liv1 → device077_LIV1.csv
Fixed: device077liv2 → device077_LIV2.csv
Fixed: device078liv1 → device078_LIV1.csv
Fixed: device078liv2 → device078_LIV2.csv
Fixed: device079liv1 → device079_LIV1.csv
Fixed: device079liv2 → device079_L

In [13]:
from collections import Counter
import os, re

files = [f for f in os.listdir("raw_LIV") if re.search(r"LIV[12]\.csv$", f, re.IGNORECASE)]
counts = Counter(re.match(r"(device\d{3})_", f).group(1) for f in files)

bad = {k:v for k,v in counts.items() if v != 2}

print("Total devices:", len(counts))
print("Devices not having exactly 2 scans:", bad)

Total devices: 106
Devices not having exactly 2 scans: {}
